In [1]:
import pandemic_simulator as ps
import random

from tf_agents.specs import BoundedArraySpec
import numpy as np


2022-04-12 16:47:42.203696: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-04-12 16:47:42.203719: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
/home/alfred/.local/lib/python3.8/site-packages/flatbuffers/compat.py:19: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  import imp
/home/alfred/.local/lib/python3.8/site-packages/keras_preprocessing/image/utils.py:23: DeprecationWarning: NEAREST is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.NEAREST or Dither.NONE instead.
  'nearest': pil_image.NEAREST,
/home/alfred/.local/lib/python3.8/site-packages/keras_preprocessing/image/utils.py:24: DeprecationWarning: BILINEAR is deprecated and will be rem

In [2]:
import base64
import IPython
import matplotlib.pyplot as plt
import os
import reverb
import tempfile

import tensorflow as tf

from tf_agents.agents.ddpg import critic_network
from tf_agents.agents.sac import sac_agent
from tf_agents.agents.sac import tanh_normal_projection_network
from tf_agents.environments import suite_pybullet
from tf_agents.metrics import py_metrics
from tf_agents.networks import actor_distribution_network
from tf_agents.policies import greedy_policy
from tf_agents.policies import py_tf_eager_policy
from tf_agents.policies import random_py_policy
from tf_agents.replay_buffers import reverb_replay_buffer
from tf_agents.replay_buffers import reverb_utils
from tf_agents.train import actor
from tf_agents.train import learner
from tf_agents.train import triggers
from tf_agents.train.utils import spec_utils
from tf_agents.train.utils import strategy_utils
from tf_agents.train.utils import train_utils

tempdir = tempfile.gettempdir()

In [3]:
env_name = "MinitaurBulletEnv-v0" # @param {type:"string"}

# Use "num_iterations = 1e6" for better results (2 hrs)
# 1e5 is just so this doesn't take too long (1 hr)
num_iterations = 100000 # @param {type:"integer"}

initial_collect_steps = 10000 # @param {type:"integer"}
collect_steps_per_iteration = 1 # @param {type:"integer"}
replay_buffer_capacity = 10000 # @param {type:"integer"}

batch_size = 256 # @param {type:"integer"}

critic_learning_rate = 3e-4 # @param {type:"number"}
actor_learning_rate = 3e-4 # @param {type:"number"}
alpha_learning_rate = 3e-4 # @param {type:"number"}
target_update_tau = 0.005 # @param {type:"number"}
target_update_period = 1 # @param {type:"number"}
gamma = 0.99 # @param {type:"number"}
reward_scale_factor = 1.0 # @param {type:"number"}

actor_fc_layer_params = (256, 256)
critic_joint_fc_layer_params = (256, 256)

log_interval = 5000 # @param {type:"integer"}

num_eval_episodes = 20 # @param {type:"integer"}
eval_interval = 10000 # @param {type:"integer"}

policy_save_interval = 5000 # @param {type:"integer"}

In [4]:
env = suite_pybullet.load(env_name)
env.reset()


pybullet build time: Mar 26 2022 03:00:52


current_dir=/home/alfred/anaconda3/envs/psrl/lib/python3.8/site-packages/pybullet_envs/bullet
urdf_root=/home/alfred/anaconda3/envs/psrl/lib/python3.8/site-packages/pybullet_data
argv[0]=
argv[0]=


/home/alfred/.local/lib/python3.8/site-packages/gym/spaces/box.py:84: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(f"Box bound precision lowered by casting to {self.dtype}")


TimeStep(
{'discount': array(1., dtype=float32),
 'observation': array([ 1.4883598e+00,  1.4746042e+00,  1.4819897e+00,  1.4703699e+00,
        1.4919385e+00,  1.4963975e+00,  1.4904151e+00,  1.4917127e+00,
        1.0659608e+00,  1.0986692e+00,  1.0862656e+00,  1.1132282e+00,
        1.0174963e+00,  9.8511410e-01,  1.0253606e+00,  1.0054207e+00,
        5.0321245e-01,  6.0991210e-01,  5.5202055e-01,  6.4239722e-01,
        4.7958934e-01,  4.4840056e-01,  4.9123308e-01,  4.8371759e-01,
       -2.1677210e-03,  8.3862455e-04, -3.0263662e-04,  9.9999726e-01],
      dtype=float32),
 'reward': array(0., dtype=float32),
 'step_type': array(0, dtype=int32)})

In [5]:
print('Observation Spec:')
print(env.time_step_spec().observation)
print('Action Spec:')
print(env.action_spec())

Observation Spec:
BoundedArraySpec(shape=(28,), dtype=dtype('float32'), name='observation', minimum=[  -3.1515927   -3.1515927   -3.1515927   -3.1515927   -3.1515927
   -3.1515927   -3.1515927   -3.1515927 -167.72488   -167.72488
 -167.72488   -167.72488   -167.72488   -167.72488   -167.72488
 -167.72488     -5.71        -5.71        -5.71        -5.71
   -5.71        -5.71        -5.71        -5.71        -1.01
   -1.01        -1.01        -1.01     ], maximum=[  3.1515927   3.1515927   3.1515927   3.1515927   3.1515927   3.1515927
   3.1515927   3.1515927 167.72488   167.72488   167.72488   167.72488
 167.72488   167.72488   167.72488   167.72488     5.71        5.71
   5.71        5.71        5.71        5.71        5.71        5.71
   1.01        1.01        1.01        1.01     ])
Action Spec:
BoundedArraySpec(shape=(8,), dtype=dtype('float32'), name='action', minimum=-1.0, maximum=1.0)


In [6]:
ps.init_globals(seed=random.randint(0,1000))

sim_config = ps.sh.small_town_config

viz = ps.viz.GymViz.from_config(sim_config=sim_config)
collect_env = ps.env.PandemicGymEnv.from_config(name='collect', sim_config=sim_config, pandemic_regulations=ps.sh.austin_regulations,done_fn=ps.env.done.ORDone(done_fns=[ps.env.done.InfectionSummaryAboveThresholdDone(summary_type=ps.env.infection_model.InfectionSummary.CRITICAL,threshold=sim_config.max_hospital_capacity*3),ps.env.done.NoPandemicDone(num_days=30)]))

ps.init_globals(seed=random.randint(0,1000))#check if this affects both envs or just one

eval_env = ps.env.PandemicGymEnv.from_config(name='eval', sim_config=sim_config, pandemic_regulations=ps.sh.austin_regulations,done_fn=ps.env.done.ORDone(done_fns=[ps.env.done.InfectionSummaryAboveThresholdDone(summary_type=ps.env.infection_model.InfectionSummary.CRITICAL,threshold=sim_config.max_hospital_capacity*3),ps.env.done.NoPandemicDone(num_days=30)]))


In [7]:
use_gpu = True #@param {type:"boolean"}

strategy = strategy_utils.get_strategy(tpu=False, use_gpu=use_gpu)

2022-04-12 16:47:48.794339: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-04-12 16:47:48.794375: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-04-12 16:47:48.794399: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (Lololica): /proc/driver/nvidia/version does not exist
2022-04-12 16:47:48.794707: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)


INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)


In [8]:
a=np.ndarray((1),dtype='float32')
a[0]=0.0
c=np.ndarray(1,dtype='float32')
a

array([0.], dtype=float32)

In [12]:
collect_env = suite_pybullet.load(env_name)
eval_env = suite_pybullet.load(env_name)
observation_spec, action_spec, time_step_spec = (spec_utils.get_tensor_specs(collect_env))


urdf_root=/home/alfred/anaconda3/envs/psrl/lib/python3.8/site-packages/pybullet_data
argv[0]=
argv[0]=
urdf_root=/home/alfred/anaconda3/envs/psrl/lib/python3.8/site-packages/pybullet_data
argv[0]=
argv[0]=


In [13]:
a=action_spec.minimum

In [14]:
a

array(-1., dtype=float32)

In [25]:
c=np.array(0,dtype='float32')
c

array(0., dtype=float32)

In [35]:
c=np.array(0,dtype='float32')
d=np.ndarray(c)

TypeError: only integer scalar arrays can be converted to a scalar index

In [75]:
observation_spec2 =BoundedArraySpec([13,],np.float32,minimum=0,maximum=1)
action_spec3=tf.TensorSpec(np.array([1],))
time_step_spec = BoundedArraySpec([13,],np.float32,minimum=0,maximum=1)

In [76]:
with strategy.scope():
  critic_net = critic_network.CriticNetwork(
        (observation_spec, action_spec),
        observation_fc_layer_params=None,
        action_fc_layer_params=None,
        joint_fc_layer_params=critic_joint_fc_layer_params,
        kernel_initializer='glorot_uniform',
        last_kernel_initializer='glorot_uniform')

In [77]:
with strategy.scope():
  actor_net = actor_distribution_network.ActorDistributionNetwork(
      observation_spec,
      action_spec3,
      continuous_projection_net=(
          tanh_normal_projection_network.TanhNormalProjectionNetwork))